In [1]:
# Libraries import
import pandas as pd
import seaborn as sns

In [2]:
# Importing files
df = pd.read_csv('rssprints_train.csv')
dfy = pd.read_csv('coordinates_train.csv')

df_test = pd.read_csv('rssprints_test.csv')
dfy_test = pd.read_csv('coordinates_test.csv')

In [3]:
# Checking data
df.head()

,-90,-62,-69,-87,-85,-87.1,-68,-86,-63,-85.1,...,100.598,100.599,100.600,100.601,100.602,100.603,100.604,100.605,100.606,100.607
0,-93,-65,-67,-85,-88,-87,-67,-86,-66,-88,...,100,100,100,100,100,100,100,100,100,100
1,-88,-64,-68,-88,-86,-90,-68,-88,-66,-86,...,100,100,100,100,100,100,100,100,100,100
2,-90,-63,-66,-87,-88,-91,-67,-86,-64,-85,...,100,100,100,100,100,100,100,100,100,100
3,100,-64,-68,-88,100,-88,-66,-88,-63,-86,...,100,100,100,100,100,100,100,100,100,100
4,100,-65,-67,-85,-87,-89,-66,-89,-65,-87,...,100,100,100,100,100,100,100,100,100,100


In [4]:
# Plotting
X_train = df
y_train = dfy
X_test = df_test
y_test = dfy_test

In [17]:
#Importing Libraries
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
import numpy as np


rmse_dict = {}    
def rmse(correct,estimated):
    rmse_val = np.sqrt(mean_squared_error(correct,estimated)) 
    return rmse_val

# Generating the Table Frame for metrics
evluation_table = pd.DataFrame({  'Model_desc':[],
                        'Model_param':[],
                        'r2_train': [],
                        'r2_test': [],
                        'rms_train':[], 
                        'rms_test': [],
                        'mae_train': [],
                        'mae_test': [],
                        'mape_train':[],
                        'mape_test':[],
                        'cross_val_score' : []})


# Evaluating the model
def evaluate_model(model, model_desc,model_param, X_train, y_train, X_test, y_test):
    global evluation_table
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
        
    print(y_train.shape,y_train_pred.shape)
    
    try:
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
    except:
        r2_train = "not calculated"
        r2_test = "not calculated"
    try:
        rms_train = rmse(y_train,y_train_pred)
        rms_test = rmse(y_test, y_test_pred)
    except:
        rms_train = "not calculated"
        rms_test = "not calculated"
    try:
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
    except:
        mae_train = "not calculated"
        mae_test = "not calculated"
    try:
        mape_train = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100
        mape_test = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100
    except:
        mape_train = "not calculated"
        mape_test = "not calculated"   
    try:
        cv_score = cross_val_score(model, X_train, y_train, cv=10)
        cv_score = cv_score.mean()
    except:
        cv_score = "Not calulated"
    model_param = pd.DataFrame({'Model_desc':[model_desc],
                            'Model_param':[model_param],
                            'r2_train': [r2_train],
                            'r2_test': [r2_test],
                            'rms_train':[rms_train], 
                            'rms_test': [rms_test],
                            'mae_train': [mae_train],
                            'mae_test': [mae_test],
                            'mape_train':[mape_train],
                            'mape_test':[mape_test],
                            'cross_val_score' : [cv_score]})

    evluation_table = evluation_table.append([model_param])
 
    return evluation_table

------------------------------------RANDOM FOREST----------------------------------------


In [18]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor() 
regressor.fit(X_train, y_train)
y_train.shape

(575, 3)

In [19]:
evaluate_model(regressor, "RandomForestRegressor",regressor,X_train,y_train.values, X_test , y_test.values)

(575, 3) (575, 3)


ValueError: continuous-multioutput is not supported

In [8]:
# test = df
mtest = pd.read_csv('mtest.csv')
# print(mtest)
print(regressor.predict(mtest))

# removetest = dfy.head()
# print(removetest)
# print(101100389 - 101100102)

[[11.81646688 26.98141389  3.5       ]]


----------------------------KNN-----------------------------------

In [9]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=200)
knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=200)

In [10]:
evaluate_model(knn, "K Nearest Neighbor",knn,X_train,y_train.values, X_test , y_test.values)

(575, 3) (575, 3)


,Model_desc,Model_param,r2_train,r2_test,rms_train,rms_test,mae_train,mae_test,mape_train,mape_test,cross_val_score
0,RandomForestRegressor,"(DecisionTreeRegressor(max_features='auto', ra...",0.989304,0.727867,0.367223,1.830723,0.210193,1.147241,2.480787,14.488813,-0.683614
0,K Nearest Neighbor,KNeighborsRegressor(n_neighbors=200),0.565777,0.542416,2.440830,2.489726,1.773635,1.842117,18.680565,19.693474,-3.961526


----------------------- LINEAR REGRESSION------------------

In [11]:
from sklearn.linear_model import LinearRegression

linear_regressor = LinearRegression().fit(X_train, y_train)

linear_regressor.score(X_train, y_train)

0.8341078220197908

In [12]:
evaluate_model(linear_regressor, "LinearRegressor",linear_regressor,X_train,y_train.values, X_test , y_test.values)

(575, 3) (575, 3)


,Model_desc,Model_param,r2_train,r2_test,rms_train,rms_test,mae_train,mae_test,mape_train,mape_test,cross_val_score
0,RandomForestRegressor,"(DecisionTreeRegressor(max_features='auto', ra...",0.989304,7.278669e-01,0.367223,1.830723e+00,0.210193,1.147241e+00,2.480787,1.448881e+01,-6.836143e-01
0,K Nearest Neighbor,KNeighborsRegressor(n_neighbors=200),0.565777,5.424157e-01,2.440830,2.489726e+00,1.773635,1.842117e+00,18.680565,1.969347e+01,-3.961526e+00
0,LinearRegressor,LinearRegression(),0.834108,-4.291913e+23,1.456876,1.740251e+12,1.002514,1.105714e+11,11.735467,8.383707e+11,-1.910328e+23


In [13]:
# test = df
mtest = pd.read_csv('mtest.csv')
# print(mtest)
print(linear_regressor.predict(mtest))

[[ 9.66902161 26.33401489  3.67459106]]


In [ ]:
from sklearn.metrics import accuracy_score
